In [1]:
import pandas as pd
import numpy as np

use_cols = [
    '인허가일자',
    '상세영업상태명',
    '폐업일자',
    '소재지전체주소',
    '도로명전체주소',
    '업태구분명',
    '좌표정보x(epsg5174)',
    '좌표정보y(epsg5174)',
    '사업장명'
]

df = pd.read_csv(
    r"C:\ai\lecNote\1st_project\data\6110000_서울특별시_07_24_04_P_일반음식점.csv",
    encoding="cp949",
    usecols=use_cols,
    low_memory=False
)


In [2]:
# 소재지 주소가 있으면 우선 사용, 없으면 도로명 주소 사용
df['주소'] = df['소재지전체주소'].fillna(df['도로명전체주소'])


In [3]:
# 주소 문자열에서 '○○구' 패턴 추출
df['구'] = df['주소'].str.extract(r'(서울특별시\s*)?(\w+구)')[1]
# 구 정보 없는 행 제거
df[['주소', '구']].head()

,주소,구
0,서울특별시 강북구 수유동 229-46,강북구
1,서울특별시 종로구 관훈동 155-2,종로구
2,서울특별시 종로구 관훈동 155-2,종로구
3,서울특별시 종로구 인사동 241,종로구
4,서울특별시 양천구 신월동 60-33,양천구


In [4]:
# '구' 결측치 제거
df['구'].isna().sum()
df = df[df['구'].notna()].copy()

In [5]:
# 서울특별시 데이터만 유지 (타 지역 자동 제거)
df = df[df['주소'].str.contains('서울', na=False)].copy()


In [6]:
# 문자열 → datetime 변환
df['인허가일자'] = pd.to_datetime(df['인허가일자'], errors='coerce')
df['폐업일자'] = pd.to_datetime(df['폐업일자'], errors='coerce')

# 인허가일자 없는 행 제거
df = df[df['인허가일자'].notna()].copy()

# ✅ 2015년 이후만 사용
df = df[df['인허가일자'] >= '2015-01-01'].copy()

# 좌표 결측치 제거
df = df[
    df['좌표정보x(epsg5174)'].notna() &
    df['좌표정보y(epsg5174)'].notna()
].copy()

In [7]:
df['업태구분명'] = df['업태구분명'].str.strip()

In [8]:
cafe_keywords = [
    '카페', '커피', 'coffee', 'COFFEE', '까페', 'cafe', 'CAFÉ', '다방', '샐러드'
]


In [9]:
food_keywords = [
    '식당', '음식점', '밥상', '한식', '순대', '곱창', '국수', '고기'
]


In [20]:
soolzip_keywords = [
    '술집','주점','술', '호프', '건배'
]


In [ ]:
fried_keywords = [
    '치킨'
]


In [21]:
def has_cafe_keyword(name):
    if pd.isna(name):
        return 0
    
    return int(any(kw in name for kw in cafe_keywords))


In [22]:
def has_food_keyword(name):
    if pd.isna(name):
        return 0
    
    return int(any(kw in name for kw in food_keywords))


In [23]:
def has_soolzip_keyword(name):
    if pd.isna(name):
        return 0
    
    return int(any(kw in name for kw in soolzip_keywords))


In [24]:
df['카페_포함_기타업태'] = (
    (df['사업장명'].apply(has_cafe_keyword) == 1) &
    (df['업태구분명'] == '기타')
).astype(int)


In [ ]:
df['식당_포함_기타업태'] = (
    (df['사업장명'].apply(has_food_keyword) == 1) &
    (df['업태구분명'] == '기타')
).astype(int)


In [ ]:
df['술집_포함_기타업태'] = (
    (df['사업장명'].apply(has_soolzip_keyword) == 1) &
    (df['업태구분명'] == '기타')
).astype(int)


In [25]:
df['식당_포함_기타업태'].value_counts()

0    141787
1       603
Name: 식당_포함_기타업태, dtype: int64

In [26]:
df['카페_포함_기타업태'].value_counts()


0    136200
1      6190
Name: 카페_포함_기타업태, dtype: int64

In [27]:
df['술집_포함_기타업태'].value_counts()


0    142021
1       369
Name: 술집_포함_기타업태, dtype: int64

In [93]:
KEYWORD_GROUPS = {
    '카페': ['카페', '커피', 'coffee', 'COFFEE','케이크' '까페', 'cafe', 'CAFÉ', '다방', '샐러드', '프레소', '티', '차', '디저트'],
    '한식': ['식당', '찜', '전골', '전' '음식점', '밥상', '한식', '순대', '찌개', '구이' '곱창', '국수', '고기', '구이', '국', '밥', '마당', '옥', '관', '떡볶이', '김밥'],
    '치킨,주점': ['크리스피', 'KFC', 'kfc', '치킨', '비비큐', 'bbq',  'BBQ', '후라이드', '술집', '주점', '포장마차', '술', '호프', '건배', '맥주', '포차','펍','pup' ,'PUP', '주점', '바', '룸' ,'와인', '포차','소주', '맥주', '아지트', '잔', '위스키', '클럽'],
    '양식,외식': ['베이커리', '빵', '아웃백'],
    '중식,일식': ['마라탕','양꼬치','샤브']
}


In [96]:
def has_keyword_other_type(name, category, 업태):
    if 업태 != '기타':
        return 0
    if pd.isna(name):
        return 0
    
    keywords = KEYWORD_GROUPS.get(category, [])
    return int(any(kw in name for kw in keywords))


In [99]:
df['카페_포함_기타업태'] = df.apply(
    lambda r: has_keyword_other_type(r['사업장명'], '카페', r['업태구분명']),
    axis=1
)

df['한식_포함_기타업태'] = df.apply(
    lambda r: has_keyword_other_type(r['사업장명'], '한식', r['업태구분명']),
    axis=1
)

df['치킨,주점_포함_기타업태'] = df.apply(
    lambda r: has_keyword_other_type(r['사업장명'], '치킨,주점', r['업태구분명']),
    axis=1
)

df['양식,외식_포함_기타업태'] = df.apply(
    lambda r: has_keyword_other_type(r['사업장명'], '양식,외식', r['업태구분명']),
    axis=1
)

df['중식,일식_포함_기타업태'] = df.apply(
    lambda r: has_keyword_other_type(r['사업장명'], '중식,일식', r['업태구분명']),
    axis=1
)

In [100]:
df[['카페_포함_기타업태', '한식_포함_기타업태', '치킨,주점_포함_기타업태', '양식,외식_포함_기타업태', '중식,일식_포함_기타업태']].sum()


카페_포함_기타업태       7767
한식_포함_기타업태       1886
치킨,주점_포함_기타업태    3421
양식,외식_포함_기타업태     392
중식,일식_포함_기타업태     150
dtype: int64

In [ ]:
# 2015 ~ 2025 & 업태구분명 = 기타
df_etc_2015_2025 = df[
    (df['업태구분명'] == '기타') &
    (df['인허가일자'] >= '2015-01-01') &
    (df['인허가일자'] <= '2025-12-31')
].copy()

# 상호명만 추출 (결측 제거 + 중복 제거)
etc_shop_names = (
    df_etc_2015_2025['사업장명']
    .dropna()
    .astype(str)
    .str.strip()
    .drop_duplicates()
    .reset_index(drop=True)
)

etc_shop_names.head(), len(etc_shop_names)


In [ ]:
csv_path = r"C:\ai\lecNote\1st_project\data\서울_일반음식점_기타업태_상호명_2015_2025.csv"

etc_shop_names.to_csv(
    csv_path,
    index=False,
    encoding="utf-8-sig",
    header=['사업장명']
)

print(f"저장 완료: {csv_path}")
